In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

COMPOSICIÓN PORTAFOLIO

In [ ]:
def solicitar_portafolio():
    print("Ingrese los activos de su portafolio y la ponderación de cada uno (en porcentaje).")
    print("Ejemplo: AAPL; 20")
    print("La suma de las ponderaciones no debe superar el 100%.\n")
    print("IMPORTANTE: El ticker debe estar disponible en Yahoo Finance, ya que es la API utilizada para obtener precios.\n")

    activos = []
    suma_ponderaciones = 0.0

    while True:
        activo = input("Nombre del activo (o 'fin' para terminar): ").strip().upper()
        if activo.lower() == 'fin':
            break

        # Validación de ticker en Yahoo Finance
        try:
            test = yf.Ticker(activo)
            hist = test.history(period="1d")
            if hist.empty:
                print(f"El ticker '{activo}' no está disponible o no tiene datos en Yahoo Finance. Por favor, ingrese un ticker válido.")
                continue
        except Exception as e:
            print(f"Error al comprobar el ticker '{activo}': {e}")
            continue

        try:
            ponderacion = float(input(f"Ponderación de {activo} (%): "))
        except ValueError:
            print("Por favor ingrese un número válido para la ponderación.")
            continue

        if ponderacion <= 0 or ponderacion > 100:
            print("La ponderación debe ser mayor a 0 y menor o igual a 100.")
            continue

        if suma_ponderaciones + ponderacion > 100:
            print(f"La suma de las ponderaciones excedería el 100%. Actualmente: {suma_ponderaciones}%.")
            continue

        activos.append((activo, ponderacion))
        suma_ponderaciones += ponderacion
        print(f"Activo agregado: {activo} con ponderación {ponderacion}%. Suma actual: {suma_ponderaciones}%.")

        if suma_ponderaciones == 100:
            print("Has alcanzado el 100% de ponderación en tu portafolio.")
            break

    if suma_ponderaciones == 0:
        print("No se ingresaron activos válidos.")
        return []

    print("\nPortafolio final:")
    for activo, ponderacion in activos:
        print(f"- {activo}: {ponderacion}%")
    return activos

portafolio = solicitar_portafolio()

EXTRACCIÓN DE DATOS YAHOO FINANCE

In [ ]:
# Extrae los símbolos del portafolio
simbolos = [a[0].upper() for a in portafolio]

# Descarga precios históricos ajustados (solo columna 'Close')
datos = yf.download(simbolos, start='1900-01-01', progress=False, auto_adjust=True)['Close']

# Si solo hay un activo, convierte a DataFrame
if isinstance(datos, pd.Series):
    datos = datos.to_frame()

# Calcula los rendimientos diarios (returns)
retornos = datos.pct_change().dropna()

# Muestra los primeros valores
retornos.head()

CALCULO DE VOLATILIDAD (RIESGO) DE CADA ACTIVO

In [ ]:
# Calcula el desvío estándar anual de cada activo en porcentaje
desvios_anuales = retornos.std() * (252 ** 0.5) * 100

print("Desvío estándar anual (volatilidad) de cada activo (%):")
for ticker, vol in desvios_anuales.round(2).items():
    print(f"{ticker}: {vol:.2f}%")

BETA DE CADA ACTIVO

In [ ]:
print("Beta de cada activo según Yahoo Finance:")

for ticker in simbolos:
    info = yf.Ticker(ticker).info
    beta = info.get("beta", None)
    if beta is not None:
        print(f"{ticker}: {beta:.2f}")
    else:
        print(f"{ticker}: Beta no disponible")

CALCULO DEL COEFICIENTE DE CORRELACION

In [ ]:
# Limita la visualización de decimales en pandas a 4
pd.options.display.float_format = '{:.4f}'.format

# Calcula la matriz de correlación
matriz_correlacion = retornos.corr()
np.fill_diagonal(matriz_correlacion.values, np.nan)

# Función para resaltar máximo y mínimo por columna
def highlight_max_min_col(col):
    max_val = np.nanmax(col.values)
    min_val = np.nanmin(col.values)
    return [
        'background-color: #00ff00' if v == max_val else
        'background-color: #ff0000' if v == min_val else
        ''
        for v in col.values
    ]

# Aplica el estilo por columnas
matriz_correlacion.style.apply(highlight_max_min_col, axis=0)

CALCULO DE VOLATILIDAD (RIESGO) DE PORTAFOLIO

In [ ]:
# Calcula la matriz de covarianzas
matriz_covarianza = retornos.cov()

# Obtiene las ponderaciones como proporciones (no porcentaje)
ponderaciones = np.array([a[1] / 100 for a in portafolio])

# Calcula la varianza y volatilidad diaria del portafolio
varianza_portafolio = np.dot(ponderaciones.T, np.dot(matriz_covarianza, ponderaciones))
volatilidad_diaria = np.sqrt(varianza_portafolio)

# Anualiza la volatilidad (usando 252 días hábiles)
volatilidad_anual = volatilidad_diaria * np.sqrt(252)

# Muestra la volatilidad anual en términos porcentuales
print(f"Volatilidad anual del portafolio: {volatilidad_anual * 100:.2f}%")